Minimum 16 GB VRAM required to run

In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
from datasets import load_dataset
import transformers
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

In [ ]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
    )

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

print(model.get_memory_footprint())

In [ ]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
prompt_text = """
Question: What strategies can help in retaining employees?
Source1: reputation is the best thing that you can have if you have given enough to people you will already\nhave earned their trust  that you don't need to sell they will just say I want to work with you and you\njust accept it and you say sure the give give first until they ask is the long-term strategy that\ncompounds unto itself because your business will compound as a result of the Goodwill that\ncompounds in the marketplace whereas if you have a sales driven Org the only way that that\nbusiness compounds is if the sales guys compounding their recruiting on your behalf rather than\nbecause if they have a great product they have a great sales system\ngenerational wealth is transferred through education not assets ninety percent of the time by Third\nGeneration The Wealth has been squandered because they never learned how to make the money\nall they had was they were given the money and it's a lot  if you give a man a fish he eats for a day\neven if you have lots of fish you still need to teach them how to fish because that generation needs\nto teach the Next Generation how to fish because the fish do eventually run out and  on that long\ntime Horizon it's education not the asset itself that transfers it over the Long Haul\nhere's five things you can do to get anyone to understand what you're trying to tell the first is you\ntake out adverbs  when people say really very extremely it  adds words and makes it more confusing\nthe second thing is speaking the passive tone a business is valued by instead of saying that you say\nwe value a business this way  it goes from passive to active voice the third way is by using shorter\nwords and shorter sentences instead of having colons and commas just put the period there and\nstart the next sentence the fourth thing is using shorter and smaller words people have to\nunderstand what you're  saying and the last thing is you want to use vocabulary that everyone\nunderstands  rather than saying are multiple different sources i might say lots of stuff\nbe willing to negotiate everything except for your values I used to think of negotiating as a zero-sum\ngame and it's not at all you can find a way where both parties get better than they were before which\nis the entire point that is how a negotiating works if you're doing it well I didn't understand that I
Source2: you through the three values that we have at acquisition.com and why we believe them and  from an\noverarching perspective the way to come to values  is to look at what are the non-negotiables okay\nand  what i mean by that is and every company is different because the core thing about values is\nthat they have to be things that are true and innate to you right  for example at southwest  have fun\nis one of their core values and if someone does not want to have fun or does not believe in the\nprocesses that they do to have a good time then that is a non-negotiable for them that person\ncannot work at that company it is a fireable offense and  these are not aspirational these are not\nthings that you would  to have these are things that are core to who you are as a person and as a\nteam all right and  when you decide on these non-negotiables they are by their very nature\nnon-negotiable which means that you hire based on these you fire based on these and these are the\ncore spirit of the team and some of the mistakes that i see when people make values is that one\nthey don't draw the line in the sand and  one of the core things about a value is that it has to you\nhave to be able to say this is not this right you have to be able to say okay between justice and\nmercy we lean towards justice or we lean towards mercy which means you have to repel people with\nyour values all right and  if your values do not repel people then they are not values they are\nplatitudes okay and  it's very important the next thing is that when you're making your values the\nvalues themselves need to be ideally said in words that you would normally say  if you have little\nsayings inside of your community then then a lot of times those can become some of the values you\nhave  for example jim launch one of our first portfolio companies  speed is king do the boring work\nthese were different ways i mean we could just say work ethic but that's not the way we would have\nsaid it we could have say be fast or fast turnaround times but speed is king was the way we would\nhave said it okay and  some of the things that i've learned with this is also you cannot have too\nmany values as in i'll say it differently you can't have too many values you have to be very selective\nat the the true core values of the company all right and  if you're  between two a lot of times you just\nhave to chunk up  and kind of get a broader value but they become the core and i think that what we\nhave found is that three is the sweet spot and that number has continued to distill down over time\nbut i think three is the amount that your human brain can comprehend as lenses to make a decision
Task: Based on the source1 text and source2 text, answer the question.
""".strip()

# Tokenize the input
encoding = tokenizer(prompt_text, return_tensors="pt")

# Generate a response
with torch.no_grad():
    outputs = model.generate(
        input_ids=encoding["input_ids"],
        attention_mask=encoding["attention_mask"],
        max_length=1300,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.7
    )

# Decode and print the generated response
generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_response)

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset("json", data_files = "prompt_response_pairs.json")

In [ ]:
def generate_prompt(data_point):
    prompt_text = f"""
    Question: {data_point["Prompt"]}
    Source1: {data_point["Source1"]}
    Source2: {data_point["Source2"]}
    Task: Based on the source1 text and source2 text, answer the question.
    Answer: {data_point["Response"]}
    """.strip()
    return prompt_text

def generate_and_tokenize_prompt(data_point):
  prompt_text = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(prompt_text, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=2,
      gradient_accumulation_steps=4,
      num_train_epochs=2,
      learning_rate=2e-3,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "username/tunedpersona-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt_text = """
Question: What strategies can help in retaining employees?
Source1: reputation is the best thing that you can have if you have given enough to people you will already\nhave earned their trust  that you don't need to sell they will just say I want to work with you and you\njust accept it and you say sure the give give first until they ask is the long-term strategy that\ncompounds unto itself because your business will compound as a result of the Goodwill that\ncompounds in the marketplace whereas if you have a sales driven Org the only way that that\nbusiness compounds is if the sales guys compounding their recruiting on your behalf rather than\nbecause if they have a great product they have a great sales system\ngenerational wealth is transferred through education not assets ninety percent of the time by Third\nGeneration The Wealth has been squandered because they never learned how to make the money\nall they had was they were given the money and it's a lot  if you give a man a fish he eats for a day\neven if you have lots of fish you still need to teach them how to fish because that generation needs\nto teach the Next Generation how to fish because the fish do eventually run out and  on that long\ntime Horizon it's education not the asset itself that transfers it over the Long Haul\nhere's five things you can do to get anyone to understand what you're trying to tell the first is you\ntake out adverbs  when people say really very extremely it  adds words and makes it more confusing\nthe second thing is speaking the passive tone a business is valued by instead of saying that you say\nwe value a business this way  it goes from passive to active voice the third way is by using shorter\nwords and shorter sentences instead of having colons and commas just put the period there and\nstart the next sentence the fourth thing is using shorter and smaller words people have to\nunderstand what you're  saying and the last thing is you want to use vocabulary that everyone\nunderstands  rather than saying are multiple different sources i might say lots of stuff\nbe willing to negotiate everything except for your values I used to think of negotiating as a zero-sum\ngame and it's not at all you can find a way where both parties get better than they were before which\nis the entire point that is how a negotiating works if you're doing it well I didn't understand that I
Source2: you through the three values that we have at acquisition.com and why we believe them and  from an\noverarching perspective the way to come to values  is to look at what are the non-negotiables okay\nand  what i mean by that is and every company is different because the core thing about values is\nthat they have to be things that are true and innate to you right  for example at southwest  have fun\nis one of their core values and if someone does not want to have fun or does not believe in the\nprocesses that they do to have a good time then that is a non-negotiable for them that person\ncannot work at that company it is a fireable offense and  these are not aspirational these are not\nthings that you would  to have these are things that are core to who you are as a person and as a\nteam all right and  when you decide on these non-negotiables they are by their very nature\nnon-negotiable which means that you hire based on these you fire based on these and these are the\ncore spirit of the team and some of the mistakes that i see when people make values is that one\nthey don't draw the line in the sand and  one of the core things about a value is that it has to you\nhave to be able to say this is not this right you have to be able to say okay between justice and\nmercy we lean towards justice or we lean towards mercy which means you have to repel people with\nyour values all right and  if your values do not repel people then they are not values they are\nplatitudes okay and  it's very important the next thing is that when you're making your values the\nvalues themselves need to be ideally said in words that you would normally say  if you have little\nsayings inside of your community then then a lot of times those can become some of the values you\nhave  for example jim launch one of our first portfolio companies  speed is king do the boring work\nthese were different ways i mean we could just say work ethic but that's not the way we would have\nsaid it we could have say be fast or fast turnaround times but speed is king was the way we would\nhave said it okay and  some of the things that i've learned with this is also you cannot have too\nmany values as in i'll say it differently you can't have too many values you have to be very selective\nat the the true core values of the company all right and  if you're  between two a lot of times you just\nhave to chunk up  and kind of get a broader value but they become the core and i think that what we\nhave found is that three is the sweet spot and that number has continued to distill down over time\nbut i think three is the amount that your human brain can comprehend as lenses to make a decision
Task: Based on the source1 text and source2 text, answer the question.
""".strip()


encoding = tokenizer(prompt_text, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))